# <center> 3) Preparación de los Datos <center>

## Selección de Datos

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, HTML
# !(pip install chainladder)
import chainladder as cl

print("pandas: " + pd.__version__)
print("numpy: " + np.__version__)
print("chainladder: " + cl.__version__)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/john/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_12505/1933557399.py", line 6, in <cell line: 6>
    import chainladder as cl
  File "/home/john/anaconda3/lib/python3.9/site-packages/chainladder/__init__.py", line 37, in <module>
    from chainladder.utils import *  # noqa (API Import)
  File "/home/john/anaconda3/lib/python3.9/site-packages/chainladder/utils/__init__.py", line 4, in <module>
    from chainladder.utils.weighted_regression import (
  File "/home/john/anaconda3/lib/python3.9/site-packages/chainladder/utils/weighted_regression.py", line 5, in <module>
    from chainladder.utils.sparse import sp
  File "/home/john/anaconda3/lib/python3.9/site-packages/chainladder/utils/sparse.py", line 5, in <module>
    import sparse
  File "/home/john/anaconda3/lib/python3.9/site-packages/sparse/__init__.py", lin

In [ ]:
data=pd.read_csv('wkcomp_pos.csv')